<a href="https://colab.research.google.com/github/pabloecheg/Business-Intelligence/blob/main/PARCIAL_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sqlite3
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
restaurante = pd.read_excel('/content/drive/MyDrive/INTRO BI/PARCIAL 1/Restaurantes.xlsx', sheet_name= None, header=0)
futbol = pd.read_excel('/content/drive/MyDrive/INTRO BI/PARCIAL 1/PARTIDOS.xlsx', sheet_name= None, header=0)

Miramos los nombres de las tablas para cada archivo

In [ ]:
restaurante.keys()

dict_keys(['CARTA', 'PEDIDOS'])

In [ ]:
futbol.keys()

dict_keys(['EQUIPOS', 'PARTIDOS'])

Pregunta 1:

Para el restaurante de comida saludable que solo tiene domicilios, elegí dos tablas: La carta y los pedidos.

La tabla de Carta tiene columnas como ID Producto, Nombre, Descripcion, Precio y Disponible.
*   ID Producto permite tener una identificacion unica para cada producto
*   El nombre del producto es un elemento clave, junto a la descripcion que permiten saber que es cada producto y que contiene
*  El precio nos permite saber cuanto vale cada producto y que ingresos nos genera
*   Y el disponible nos permite saber si el producto esta disponible en inventario o no

La tabla de Pedidos tiene columnas como # de Pedido, Nombre del Cliente, ID Producto, Cantidad, Ciudad, Barrio y Estado.
*  El numero del pedido es el identificador unico de cada pedido
*  El nombre del cliente nos permite saber cual cliente pidio
*  Cantidad permite saber la cantidad de cada producto que el cliente pidio
*  El ID del producto permite mantener una relacion con la otra tabla
*  La ciudad permite saber desde que ciudad nos estan haciendo los pedidos y se separan por barrio
*  El estado del pedido nos permite saber si el domicilio ya fue entregado o esta pendiente




## RESTAURANTE

Creamos la base de datos

In [ ]:
con = sqlite3.connect('Restaurante.db')
cursor = con.cursor()

Pasamos la informacion del diccionario a la base de datos

In [ ]:
for hoja, restaurante in restaurante.items():
  restaurante.to_sql(hoja, con, if_exists='replace', index=False)

In [ ]:
#Para confirmar los datos
con.commit()

In [ ]:
#Visualizamos los nombres de las tablas en formato consulta
consulta = ''' SELECT name
               FROM sqlite_master
               WHERE type = 'table' '''
#Ejecutamos la consulta
cursor.execute(consulta)
#Printeamos el resultado de la consulta
print(cursor.fetchall())

[('CARTA',), ('PEDIDOS',)]


Para mirar la informacion de la tabla de CARTA

In [ ]:
consulta = ''' SELECT *
              FROM "CARTA" '''
#Ejecutamos la consulta
cursor.execute(consulta)
#Traemos el resultado de la consulta
Carta = cursor.fetchall()
for fila in Carta:
  print(fila)

(1, 'Ensalada Cesar', 'Ensalada Clasica con Lechuga, Tomate y Crotones', 28000, 'SI')
(2, 'Healthy Wrap', 'Wrap de Pollo con Vegetales', 19500, 'SI')
(3, 'Veggie Burger', 'Hamburguesa Vegetariana con mix de legumbres  servida en pan integral', 38000, 'SI')
(4, 'Salmon Horneado', 'Salmon fresco horneado al punto, sazonado con hierbas', 52000, 'SI')
(5, 'Detox Smoothie', 'Smoothie de frutas y espinaca', 14900, 'SI')


Para Mirar informacion de la tabla PEDIDOS

In [ ]:
consulta = ''' SELECT *
              FROM "PEDIDOS" '''
#Ejecutamos la consulta
cursor.execute(consulta)
#Traemos el resultado de la consulta
Pedidos = cursor.fetchall()
for fila in Pedidos:
  print(fila)

(1, 'Pablo Echeverri', 1, 2, 'Medellin', 'La Florida', 'Pendiente')
(2, 'Camilo Guzman', 2, 1, 'Medellin', 'La Calera', 'Entregado')
(3, 'Juan Escobar', 3, 4, 'Medellin', 'Los Balsos', 'Entregado')
(4, 'Juan Jose Arbelaez', 4, 2, 'Envigado', 'La Calera', 'Pendiente')
(5, 'Juan Jose Uribe', 5, 1, 'Envigado', 'Las Palmas', 'Entregado')


Lo pasamos a pandas para observarlo mejor

In [ ]:
columnas = [descripcion[0] for descripcion in cursor.description]
df_carta = pd.DataFrame(Carta, columns=columnas)
df_carta

,ID Producto,Nombre,Descripcion,Precio,Disponible
0,1,Ensalada Cesar,"Ensalada Clasica con Lechuga, Tomate y Crotones",28000,SI
1,2,Healthy Wrap,Wrap de Pollo con Vegetales,19500,SI
2,3,Veggie Burger,Hamburguesa Vegetariana con mix de legumbres ...,38000,SI
3,4,Salmon Horneado,"Salmon fresco horneado al punto, sazonado con ...",52000,SI
4,5,Detox Smoothie,Smoothie de frutas y espinaca,14900,SI


In [ ]:
columnas = [descripcion[0] for descripcion in cursor.description]
df_pedidos = pd.DataFrame(Pedidos, columns=columnas)
df_pedidos

,# Pedido,Nombre Cliente,ID Producto,Cantidad,Ciudad,Barrio,Estado
0,1,Pablo Echeverri,1,2,Medellin,La Florida,Pendiente
1,2,Camilo Guzman,2,1,Medellin,La Calera,Entregado
2,3,Juan Escobar,3,4,Medellin,Los Balsos,Entregado
3,4,Juan Jose Arbelaez,4,2,Envigado,La Calera,Pendiente
4,5,Juan Jose Uribe,5,1,Envigado,Las Palmas,Entregado


### CONSULTAS

Para mirar que pedidos estan pendientes de entrega

In [ ]:
consulta = ''' SELECT *
               FROM "PEDIDOS"
               WHERE "Estado" = "Pendiente" '''

#Ejecutamos la consulta
cursor.execute(consulta)
#Traemos el resultado de la consulta
Pedidos = cursor.fetchall()
for fila in Pedidos:
  print(fila)

(1, 'Pablo Echeverri', 1, 2, 'Medellin', 'La Florida', 'Pendiente')
(4, 'Juan Jose Arbelaez', 4, 2, 'Envigado', 'La Calera', 'Pendiente')


Para mirar que pedidos vienen de la ciudad de Medellín

In [ ]:
consulta = ''' SELECT *
               FROM "PEDIDOS"
               WHERE "Ciudad" = "Medellin" '''

#Ejecutamos la consulta
cursor.execute(consulta)
#Traemos el resultado de la consulta
Pedidos = cursor.fetchall()
for fila in Pedidos:
  print(fila)

(1, 'Pablo Echeverri', 1, 2, 'Medellin', 'La Florida', 'Pendiente')
(2, 'Camilo Guzman', 2, 1, 'Medellin', 'La Calera', 'Entregado')
(3, 'Juan Escobar', 3, 4, 'Medellin', 'Los Balsos', 'Entregado')


Para mirar cuanto ha gastado cada cliente

In [ ]:
consulta = ''' SELECT "Nombre Cliente",
               SUM(Precio * Cantidad) AS "Gasto Total"
               FROM "PEDIDOS"
               JOIN "CARTA" ON "PEDIDOS"."ID Producto" = "CARTA"."ID Producto"
               GROUP BY "Nombre Cliente"
               ORDER BY "Gasto Total" DESC '''


# Ejecutamos la consulta
cursor.execute(consulta)

# Traemos el resultado de la consulta
Pedidos = cursor.fetchall()

# Asegúrate de imprimir todas las filas
for fila in Pedidos:
    print(fila)


('Juan Escobar', 152000)
('Juan Jose Arbelaez', 104000)
('Pablo Echeverri', 56000)
('Camilo Guzman', 19500)
('Juan Jose Uribe', 14900)


Pedidos totales por Ciudad

In [ ]:
consulta = ''' SELECT "Ciudad",
               COUNT(*) AS "Pedidos Totales"
               FROM "PEDIDOS"
               GROUP BY "Ciudad"
               ORDER BY "Pedidos Totales" DESC '''


#Ejecutamos la consulta
cursor.execute(consulta)
#Traemos el resultado de la consulta
Pedidos = cursor.fetchall()
for fila in Pedidos:
  print(fila)

('Medellin', 3)
('Envigado', 2)


In [ ]:
cursor.close()
con.close()

## PARTIDOS

Creamos la Base de Datos

In [ ]:
con = sqlite3.connect('Partidos.db')
cursor = con.cursor()

Pasamos la informacion del diccionario a la base de datos

In [ ]:
for hoja, futbol in futbol.items():
  futbol.to_sql(hoja, con, if_exists='replace', index=False)

In [ ]:
#Para confirmar los cambios
con.commit()

Visualizamos los nombres de las tablas

In [ ]:
consulta = ''' SELECT name
               FROM sqlite_master
               WHERE type = 'table' '''
#Ejecutamos la consulta
cursor.execute(consulta)
#Printeamos el resultado de la consulta
print(cursor.fetchall())

[('EQUIPOS',), ('PARTIDOS',)]


Miramos la Informacion de Las Tablas en Pandas

In [ ]:
consulta = ''' SELECT *
              FROM "EQUIPOS" '''
#Ejecutamos la consulta
cursor.execute(consulta)
#Traemos el resultado de la consulta
Equipos = cursor.fetchall()
for fila in Equipos:
  print(fila)

(1, 'Deportivo Independiente Medellin', 'Medellin', 'Juan Escobar', 1)
(2, 'Atletico Nacional', 'Medellin', 'Camilo Guzman', 5)
(3, 'America de Cali', 'Cali', 'Juan Jose Arbelaez', 3)
(4, 'Millonarios', 'Bogota', 'Jacobo Velez', 2)
(5, 'Junior', 'Barranquilla', 'Santiago Gallego', 4)


In [ ]:
columnas = [descripcion[0] for descripcion in cursor.description]
df_equipos = pd.DataFrame(Equipos, columns=columnas)
df_equipos

,ID Equipo,Nombre Equipo,Ciudad,Entrenador,Titulos Ganados
0,1,Deportivo Independiente Medellin,Medellin,Juan Escobar,1
1,2,Atletico Nacional,Medellin,Camilo Guzman,5
2,3,America de Cali,Cali,Juan Jose Arbelaez,3
3,4,Millonarios,Bogota,Jacobo Velez,2
4,5,Junior,Barranquilla,Santiago Gallego,4


In [ ]:
consulta = ''' SELECT *
              FROM "PARTIDOS" '''
#Ejecutamos la consulta
cursor.execute(consulta)
#Traemos el resultado de la consulta
Partidos = cursor.fetchall()
for fila in Partidos:
  print(fila)

(1, 'America de Cali', 3, 'Atletico Nacional', 2, 'Cali', '1-2')
(2, 'Deportivo Independiente Medellin', 1, 'Millonarios', 4, 'Medellin', '2-1')
(3, 'Millonarios', 4, 'Atletico Nacional', 2, 'Bogota', '3-2')
(4, 'Junior', 5, 'Deportivo Independiente Medellin', 1, 'Barranquilla', '4-1')
(5, 'Atletico Nacional', 2, 'Millonarios', 4, 'Medellin', '1-1')


In [ ]:
columnas = [descripcion[0] for descripcion in cursor.description]
df_partidos = pd.DataFrame(Partidos, columns=columnas)
df_partidos

,ID Partido,Equipo Local,ID Equipo Local,Equipo Visitante,ID Equipo Visitante,Lugar Partido,Resultado
0,1,America de Cali,3,Atletico Nacional,2,Cali,1-2
1,2,Deportivo Independiente Medellin,1,Millonarios,4,Medellin,2-1
2,3,Millonarios,4,Atletico Nacional,2,Bogota,3-2
3,4,Junior,5,Deportivo Independiente Medellin,1,Barranquilla,4-1
4,5,Atletico Nacional,2,Millonarios,4,Medellin,1-1


### CONSULTAS

Para mirar que equipo tiene mas titulos ganados

In [ ]:
consulta = ''' SELECT "Nombre Equipo",
               SUM ("Titulos Ganados") AS "Titulos Ganados"
               FROM "EQUIPOS"
               GROUP BY "Nombre Equipo"
               ORDER BY "Titulos Ganados" DESC '''

#Ejecutamos la consulta
cursor.execute(consulta)
#Traemos el resultado de la consulta
Equipos = cursor.fetchall()
for fila in Equipos:
  print(fila)

('Atletico Nacional', 5)
('Junior', 4)
('America de Cali', 3)
('Millonarios', 2)
('Deportivo Independiente Medellin', 1)


Para mirar que ciudad tiene mas equipos

In [ ]:
consulta = ''' SELECT "Ciudad",
               COUNT(*) AS "Equipos"
               FROM "EQUIPOS"
               GROUP BY "Ciudad"
               ORDER BY "Equipos" DESC '''

#Ejecutamos la consulta
cursor.execute(consulta)
#Traemos el resultado de la consulta
Equipos = cursor.fetchall()
for fila in Equipos:
  print(fila)

('Medellin', 2)
('Cali', 1)
('Bogota', 1)
('Barranquilla', 1)


Para mirar cantidad total de partidos jugados como visitante

In [ ]:
consulta = ''' SELECT "Equipo Visitante",
               COUNT(*) AS "Partidos Jugados"
               FROM "PARTIDOS"
               GROUP BY "Equipo Visitante"
               ORDER BY "Partidos Jugados" DESC '''

#Ejecutamos la consulta
cursor.execute(consulta)
#Traemos el resultado de la consulta
Partidos = cursor.fetchall()
for fila in Partidos:
  print(fila)

('Millonarios', 2)
('Atletico Nacional', 2)
('Deportivo Independiente Medellin', 1)


En que ciudad se jugaron mas partidos?

In [ ]:
consulta = ''' SELECT "Lugar Partido",
               COUNT(*) AS "Partidos Jugados"
               FROM "PARTIDOS"
               GROUP BY "Lugar Partido"
               ORDER BY "Partidos Jugados" DESC '''

#Ejecutamos la consulta
cursor.execute(consulta)
#Traemos el resultado de la consulta
Partidos = cursor.fetchall()
for fila in Partidos:
  print(fila)

('Medellin', 2)
('Cali', 1)
('Bogota', 1)
('Barranquilla', 1)
